## 1. Carga y exploración de Datos

In [ ]:
# Import de las principales librerías a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Descarga del Dataset
!wget -O "airbnb-listings-extract.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

In [ ]:
# Lectura de los primeros 5 registros
airbnb = pd.read_csv('./airbnb-listings-extract.csv', sep=';', decimal='.')
airbnb.head()

Podemos observar que aparentemente se ha cargado correctamente el dataset. Veamos el shape para asegurarnos que efectivamente obtuvimos todos los registros que esperábamos.

In [ ]:
# Shape del Dataset
airbnb.shape

## 2. Preprocesamiento de Datos

Viendo que obtuvimos todos los registros esperados y que la descarga ocurrió sin problema, veamos cuales son los campos de los cuales disponemos. Esto para poder identificar a simple vista algunas variables que podrían no ser útiles para el modelado.

In [ ]:
# Campos del Dataset
airbnb.columns

Muchas de la variables vistas no aportan información relevante, por lo que se eliminarán directamente.

In [ ]:
# Eliminación de columnas no relevantes del Dataset
airbnb = airbnb.drop(['ID',
                      'Listing Url',
                      'Scrape ID',
                      'Last Scraped',
                      'Medium Url',
                      'Picture Url',
                      'XL Picture Url',
                      'Host ID',
                      'Host URL',
                      'Host Name',
                      'Host Thumbnail Url',
                      'Host Picture Url',
                      'Host Neighbourhood',
                      'Weekly Price',
                      'Monthly Price',
                      'Calendar Updated',
                      'Calendar last Scraped',
                      'First Review',
                      'Last Review',
                      'Reviews per Month',
                      'Geolocation',
                      'Calculated host listings count',
                      'Host Listings Count',
                      'Host Total Listings Count',
                      'Security Deposit',
                      'Cleaning Fee',
                      'Name',
                      'Summary',
                      'Space',
                      'Description',
                      'Neighborhood Overview',
                      'Notes',
                      'Transit',
                      'Access',
                      'Interaction',
                      'House Rules',
                      'Host Location',
                      'Host About'], axis=True)

Ahora veamos que efectivamente se eliminaron las columnas no relevantes.

In [ ]:
# Campos del Dataset
airbnb.columns

Veamos el tipo de dato de los campos y si existen registros vacíos.

In [ ]:
# Información relevante
airbnb.info()

Podemos notar que hay muchos campos que aportan información duplicada, imprecisa o con demasiados registros vacíos. Esto definitivamente sería un problema para el modelado, por lo que vamos a proceder a eliminarlos.

In [ ]:
# Eliminación campos con información duplicada, imprecisa o con demasiados registros vacíos
airbnb = airbnb.drop(['Street',
                      'State',
                      'Market',
                      'Smart Location',
                      'Country',
                      'Zipcode',
                      'Host Acceptance Rate',
                      'Square Feet',
                      'Has Availability',
                      'License',
                      'Jurisdiction Names',
                      'Review Scores Value',
                      'Availability 30',
                      'Availability 60',
                      'Availability 90',
                      'Review Scores Accuracy',
                      'Review Scores Cleanliness',
                      'Review Scores Checkin',
                      'Review Scores Communication',
                      'Review Scores Location',
                      'Accommodates'], axis=True)

In [ ]:
# Campos del Dataset
airbnb.columns

Veamos la información relevante de las variables que mantenemos.

In [ ]:
airbnb.info()

Ahora trabajaremos con los missing values.

In [ ]:
# Vencindarios

airbnb['Neighbourhood'] = airbnb['Neighbourhood Group Cleansed'].fillna(airbnb['Neighbourhood Cleansed'])
airbnb = airbnb.drop(['Neighbourhood Group Cleansed', 'Neighbourhood Cleansed', 'City', 'Country Code'], axis=1)

# Textos

airbnb['Features'] = airbnb['Features'].fillna('')
airbnb['Features'] = airbnb['Features'].apply(lambda x: len(str(x).split(',')))

airbnb['Amenities'] = airbnb['Amenities'].fillna('')
airbnb['Amenities'] = airbnb['Amenities'].apply(lambda x: len(str(x).split(',')))

airbnb['Host Verifications'] = airbnb['Host Verifications'].fillna('')
airbnb['Host Verifications'] = airbnb['Host Verifications'].apply(lambda x: len(str(x).split(',')))

# Fechas

from datetime import datetime

airbnb = airbnb.dropna(subset=['Host Since'])
airbnb['Host Since'] = airbnb['Host Since'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d'))
airbnb['Host Since'] = airbnb['Host Since'].apply(lambda x: 2017-x.year)

# Host

airbnb['Host Response Time'] = airbnb['Host Response Time'].fillna(airbnb['Host Response Time'].mode()[0])
airbnb = airbnb.drop('Experiences Offered', axis=1)

Ahora verificamos nuestros campos nuevamente para asegurarnos que los cambios han sido aplicados.

In [ ]:
# Campos del Dataset
airbnb.columns

## 3. Carga de imágenes

In [ ]:
import imageio as io
import numpy as np
import cv2
n_images = 800
images = np.zeros((n_images, 224, 224, 3), dtype=np.uint8)
urls = airbnb['Thumbnail Url']

i_aux = 0
good_urls = []
for i_img, url in enumerate(urls):
    if len(good_urls) >= n_images:
        # ya tenemos n_images imágenes
        break
    try:
        img = io.imread(url)
        images[i_aux] = cv2.resize(img, (224, 224))
        good_urls.append(i_img)
        i_aux += 1
        print(f'Imagen {i_img} descargada')
        print(len(good_urls))
    except IOError as err:
        pass

In [ ]:
# Visualizamos las imágenes cargadas
print(images.shape)

In [ ]:
# Mantenemos los datos numéricos solo para aquellos pisos que tienen imágenes 
# y las hemos obtenido
airbnb = airbnb.iloc[good_urls, :]
print(airbnb.shape)

In [ ]:
# Obtener las etiquetas de regresion
y_reg = airbnb['Price']

In [ ]:
# guardamos las imágenes (y yo os recomiendo que os lo guardéis en GDrive para evitar tener que repetir esto)
np.save('images.npy', images)
np.save('final_data.npy', airbnb)

In [ ]:
# y un rango para clasificación (del 1 al 3 por ejemplo: barato, normal, caro)
plt.hist(y_reg, bins=10)
plt.show()

In [ ]:
y_class = []
for x in y_reg:
    # barato
    if x <= 50:
        y_class.append(0)
    elif x <=150:
        y_class.append(1)
    else:
        y_class.append(2)

In [ ]:
# veamos cómo ha quedado la distribución al convertirla a 3 clases
plt.hist(y_class, bins=3)
plt.show()

## 4. Normalización

Todavía quedan algunos campos con missing values, pero al ser pocos, podemos rellenarlos. Para este fin usaremos la media o la moda según corresponda.

In [ ]:
# Rellenamos algunos missing values con la media
airbnb['Host Response Rate'] = airbnb['Host Response Rate'].fillna(airbnb['Host Response Rate'].mean())
airbnb['Review Scores Rating'] = airbnb['Review Scores Rating'].fillna(airbnb['Review Scores Rating'].mean())

# Rellenamos algunos missing values con la moda
airbnb['Bathrooms'] = airbnb['Bathrooms'].fillna(airbnb['Bathrooms'].mode())
airbnb['Beds'] = airbnb['Beds'].fillna(airbnb['Beds'].mode())

# Nos aseguramos que nuestro proceso ha ocurrido satisfactoriamiente
airbnb.info()

Ahora veremos cuales son nuestras variables categóricas para codificarlas mediante OneHotEnconding.

In [ ]:
airbnb.select_dtypes(include='object').columns

In [ ]:
# Aplicamos One Hot Encoding mediante get_dummies a todas las columnas de tipo object con excepción de 'Thumbnail Url'
airbnb = pd.get_dummies(airbnb, columns=['Host Response Time',
                                         'Neighbourhood',
                                         'Property Type',
                                         'Room Type',
                                         'Bed Type',
                                         'Cancellation Policy'])

# Comprobamos que ha funcionado
airbnb.head().T

Ahora procederemos a normalizar. Para esto utilizaremos la clase MaxMinScaler de Scikit-Learn.

In [ ]:
# Importamos la clase a utilizar
from sklearn.preprocessing import MinMaxScaler

# Definimos la función normalizadora
def normalizadora(campo):
  CampoTransformar = airbnb["Host Response Rate"]
  CampoTransformar_nparray = CampoTransformar.values.reshape(-1, 1)
  scaler = MinMaxScaler()
  return scaler.fit_transform(CampoTransformar_nparray)

# Aplicamos la función normalizadora a nuestro DataFrame
airbnb["Host Response Rate"] = normalizadora("Host Response Rate")
airbnb["Host Verifications"] = normalizadora("Host Verifications")
airbnb["Latitude"] = normalizadora("Latitude")
airbnb["Longitude"] = normalizadora("Longitude")
airbnb["Bathrooms"] = normalizadora("Bathrooms")
airbnb["Bedrooms"] = normalizadora("Bedrooms")
airbnb["Beds"] = normalizadora("Beds")
airbnb["Amenities"] = normalizadora("Amenities")
airbnb["Guests Included"] = normalizadora("Guests Included")
airbnb["Extra People"] = normalizadora("Extra People")
airbnb["Minimum Nights"] = normalizadora("Minimum Nights")
airbnb["Maximum Nights"] = normalizadora("Maximum Nights")
airbnb["Availability 365"] = normalizadora("Availability 365")
airbnb["Number of Reviews"] = normalizadora("Number of Reviews")
airbnb["Features"] = normalizadora("Features")
airbnb["Review Scores Rating"] = normalizadora("Review Scores Rating")
airbnb["Host Since"] = normalizadora("Host Since")

# Comprobamos que ha funcionado
airbnb.head().T

## 5. Modelado

A partir de aquí dividiremos el DataFrame en dos versiones. Una que utilizaremos para trabajar un modelo basado en datos 1D y una que utilizaremos para trabajar un modelo basado en imágenes.

In [ ]:
# DataFrame para modelo basado en datos
airbnb_datos = airbnb.drop('Thumbnail Url', axis=1)

# DataFrame para modelo basado en imagenes
airbnb_imagenes = airbnb.copy()

#### Primer módulo: Modelo basado en datos 1D

Como buena práctica, convertiremos los datos del DataFrame con el cual vamos a trabajar a float32.

In [ ]:
airbnb_datos = airbnb_datos.astype(np.float32)
airbnb_datos

Ahora realizaremos el split en Train, Validation y Test.

In [ ]:
# Importamos la clase necesaria de Scikit-Learn para hacer el split
from sklearn.model_selection import train_test_split

# Dividimos el DataFrame en df_train_val y df_test
df_train_val, df_test = train_test_split(airbnb_datos, test_size=0.2, random_state=42)

# Dividiremos df_train_val en df_train y df_val
df_train, df_val = train_test_split(df_train_val, test_size=0.1, random_state=42)

Ahora dividiremos nuestros DataFrames en variables predictoras y variable a predecir.

In [ ]:
# df_train
x_train = df_train.drop('Price', axis=1)
y_train = df_train['Price']

# df_val
x_val = df_val.drop('Price', axis=1)
y_val = df_val['Price']

# df_test
x_test = df_test.drop('Price', axis=1)
y_test = df_test['Price']

Por último, normalizaremos nuestra variable a predecir para los datos de train, validation y test. Esto con el fin de que el entrenamiento del modelo sea menos complejo en términos computacionales. Esto lo haremos dividiendo entre el precio máximo.

In [ ]:
# Precio máximo
maxPrice = airbnb_datos['Price'].max()

# Normalización variable a predecir
y_train = y_train / maxPrice
y_val = y_val / maxPrice
y_test = y_test / maxPrice

Ahora procederemos a crear el modelo.

In [ ]:
# Importamos las clases a utilizar
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

# Definimos los hiperparámetros que utilizaremos
LearningRate = 0.05
NumeroEpocas = 100

# Definimos la arquitectura
model_1D = Sequential()
model_1D.add(Dense(32, activation='relu', input_dim=x_train.shape[1]))
model_1D.add(Dense(16, activation='relu'))
model_1D.add(Dense(8, activation='relu'))
model_1D.add(Dense(1, activation='sigmoid'))

# Compilamos y ajustamos el modelo
model_1D.compile(loss='mse', optimizer=SGD(LearningRate), metrics=['mse'])
H = model_1D.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=NumeroEpocas, shuffle=True)

Ahora veamos como son los errores.

In [ ]:
# Primero calculamos las predicciones
y_pred_train = model_1D.predict(x_train)
y_pred_val = model_1D.predict(x_val)
y_pred_test = model_1D.predict(x_test)

# Ahora desnormalizamos las predicciones
y_pred_train_desnorm = y_pred_train[:, 0] * maxPrice
y_pred_val_desnorm = y_pred_val[:, 0] * maxPrice
y_pred_test_desnorm = y_pred_test[:, 0] * maxPrice

# Luego desnormalizamos los valores reales
y_train_desnorm = y_train * maxPrice
y_val_desnorm = y_val * maxPrice
y_test_desnorm = y_test * maxPrice

# Importamos la clase que nos permitirá medir los errores
from sklearn.metrics import mean_squared_error

# Calculamos los errores
rmse_train = mean_squared_error(y_train_desnorm, y_pred_train_desnorm, squared=False)
rmse_val = mean_squared_error(y_val_desnorm, y_pred_val_desnorm, squared=False)
rmse_test = mean_squared_error(y_test_desnorm, y_pred_test_desnorm, squared=False)

# Mostramos los resultados
print(f'RMSE train: {rmse_train}')
print(f'RMSE val: {rmse_val}')
print(f'RMSE test: {rmse_test}')

# Hacemos la gráfica
epocas = range(1, len(H.history['loss']) + 1)
plt.plot(epocas, H.history['loss'])
plt.plot(epocas, H.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model 1D')
plt.show()

Podemos ver que los valores obtenidos por train, validation y test son cercanos, por lo que podemos concluir que no estamos sufriendo de Overfitting. De igual forma, podemos observar que la gráfica muestra cercania entre los valores de train y validation. Con esto podemos confirmar que la predicción es capaz de darnos valores razonablemente cercanos a la realidad.

#### Primer módulo: Modelo basado en imágenes

En primer lugar, definamos nustras variables predictoras y nuestra variable a predecir.

In [ ]:
dfx = images
dfy = airbnb_imagenes['Price']

Ahora realizaremos el split en Train, Validation y Test.

In [ ]:
# Importamos la clase necesaria de Scikit-Learn para hacer el split
from sklearn.model_selection import train_test_split

# Dividimos los DataFrames dfx y dfy en x_train_val, x_test, y_train_val y y_test
x_train_val_images, x_test_images, y_train_val_images, y_test_images = train_test_split(dfx, dfy, test_size=0.2, random_state=42)

# Dividiremos los DataFrames x_train_val y y_train_val en x_train, x_val, y_train y y_val
x_train_images, x_val_images, y_train_images, y_val_images = train_test_split(x_train_val_images,
                                                                              y_train_val_images,
                                                                              test_size=0.1,
                                                                              random_state=42)

Por último, normalizaremos nuestras variables predictoras y nuestras variables a predecir. Esto con el fin de que el entrenamiento del modelo sea menos complejo en términos computacionales.

In [ ]:
# Precio máximo
maxPrice = airbnb_datos['Price'].max()

# Normalización variable a predecir
y_train_images = y_train_images / maxPrice
y_val_images = y_val_images / maxPrice
y_test_images = y_test_images / maxPrice

# Normalización variables predictoras
x_train_images = x_train_images.astype('float32') / 255.0
x_val_images = x_val_images.astype('float32') / 255.0
x_test_images = x_test_images.astype('float32') / 255.0

Ahora procedemos a crear el modelo.

In [ ]:
# Importamos las clases a utilizar
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Definimos la arquitectura
model_images = Sequential()
model_images.add(Conv2D(256, (3,3), activation='relu', input_shape=(224,224,3)))
model_images.add(MaxPooling2D(pool_size=(2,2)))
model_images.add(Dropout(0.25))
model_images.add(Conv2D(64, (3,3), activation='relu'))
model_images.add(MaxPooling2D(pool_size=(2,2)))
model_images.add(Dropout(0.25))
model_images.add(Conv2D(16, (3,3), activation='relu'))
model_images.add(MaxPooling2D(pool_size=(2,2)))
model_images.add(Dropout(0.25))
model_images.add(Flatten())
model_images.add(Dense(8, activation='relu'))
model_images.add(Dropout(0.25))
model_images.add(Dense(1, activation='sigmoid'))

# Compilamos y ajustamos el modelo
model_images.compile(optimizer=Adam(learning_rate=0.05), loss='mse', metrics=['mse'])
H = model_images.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_val, y_val), shuffle=True)

# Evaluamos el modelo
scores = model_images.evaluate(x_test_images, y_test_images, verbose=1)
print(f'Test Loss: {scores[0]}')
print(f'Test Accuracy: {scores[1]}')

Ahora veamos como son los errores.

In [ ]:
# Primero calculamos las predicciones
y_pred_train_images = model_images.predict(x_train_images)
y_pred_val_images = model_images.predict(x_val_images)
y_pred_test_images = model_images.predict(x_test_images)

# Ahora desnormalizamos las predicciones
y_pred_train_images_desnorm = y_pred_train_images[:, 0] * maxPrice
y_pred_val_images_desnorm = y_pred_val_images[:, 0] * maxPrice
y_pred_test_images_desnorm = y_pred_test_images[:, 0] * maxPrice

# Luego desnormalizamos los valores reales
y_train_images_desnorm = y_train_images * maxPrice
y_val_images_desnorm = y_val_images * maxPrice
y_test_images_desnorm = y_test_images * maxPrice

# Importamos la clase que nos permitirá medir los errores
from sklearn.metrics import mean_squared_error

# Calculamos los errores
rmse_train = mean_squared_error(y_train_images_desnorm, y_pred_train_images_desnorm, squared=False)
rmse_val = mean_squared_error(y_val_images_desnorm, y_pred_val_images_desnorm, squared=False)
rmse_test = mean_squared_error(y_test_images_desnorm, y_pred_test_images_desnorm, squared=False)

# Mostramos los resultados
print(f'RMSE train: {rmse_train}')
print(f'RMSE val: {rmse_val}')
print(f'RMSE test: {rmse_test}')

# Hacemos la gráfica
epocas = range(1, len(H.history['loss']) + 1)
plt.plot(epocas, H.history['loss'])
plt.plot(epocas, H.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Images')
plt.show()

Estos resultados son un poco desfavorables. Sin embargo, existe mucho campo de mejora si se ajustan los hiperparámetros o la arquitectura de la red neuronal.

#### Segundo módulo: Modelo híbrido

Ahora procederemos a construir un modelo híbrido. Esto se hará con base en el modelo de datos 1D y en el modelo de imágenes.

In [ ]:
# En primer lugar, importamos las clases a utilizar
from tensorflow.keras import optimizers, Model
from tensorflow.keras.layers import concatenate, Input, GlobalMaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

# Modelo 1D
modelo1D = Dense(8, activation='relu')(Input(shape=x_train.shape[1]))
modelo1D = Dense(4, activation='relu')(modelo1D)
modelo1D = Model(inputs=Input(shape=x_train.shape[1]), outputs=modelo1D)

# Preparación Modelo basado en imágenes
pre_modelo_imagenes = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in pre_modelo_imagenes.layers:
  if layer.name == 'block5_conv1':
    break
  layer.trainable = False

# Modelo basado en imágenes
modelo_imagenes = GlobalMaxPool2D(name='GlobalMaxPool')(pre_modelo_imagenes.layers[-1].output)
modelo_imagenes = Dense(16, activation='relu', name='fc1')(modelo_imagenes)
modelo_imagenes = Dense(8, activation='relu', name='fc2')(modelo_imagenes)
modelo_imagenes = Model(inputs=pre_modelo_imagenes.inputs, outputs=modelo_imagenes)

# Combinación de modelos
modelo_combinado = concatenate([modelo1D.output, modelo_imagenes.output])
capa = Dense(2, activation='relu')(modelo_combinado)
capa = Dense(1, activation='sigmoid')(capa)
HybridModel = Model(inputs=[modelo1D.input, modelo_imagenes.input], outputs=capa)

# Compilación y ajuste del modelo
HybridModel.compile(optimizer=Adam(learning_rate=0.005), loss='mse', metrics=['mse'])
H = HybridModel.fit([x_train, x_train_images], y_train, epochs=20, batch_size=64, validation_data=([x_val, x_val_images], y_val))

# Graficamos las pérdidas
epocas = range(1, len(H.history['loss']) + 1)
plt.plot(epocas, H.history['loss'])
plt.plot(epocas, H.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Hybrid Model')
plt.show()

Podemos concluir que el potencial que poseen las redes neuronales es muy interesante. Actualmente, los únicos problemas que estos presentan son una complejidad computacional muy elevada y un largo tiempo de espera para entrenamiento. Si bien es cierto que esto es una desventaja para el Deep Learning, la tecnología va en mejora y, más pronto que tarde, los equipos serán más potentes y asequibles. Eventualmente la complejidad computacional no será un problema.